
• Total forecast sales for Q4 of 2012 (where Q4
includes October, November, and December of 2012)




Please be sure to explain how you performed these
predictions and what (if any) limitations or risks there
may be in the predicted values

In [23]:
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from sklearn import metrics
import numpy as np

In [9]:
data = pd.read_csv('Sales by Store - Department.xls',sep='\t')

In [10]:
data['Month'] = pd.DatetimeIndex(data['Date']).month

In [11]:
data['Quarter'] = pd.DatetimeIndex(data['Date']).quarter

In [12]:
data['Year'] = pd.DatetimeIndex(data['Date']).year

data['Week'] = pd.DatetimeIndex(data['Date']).week

In [24]:
monthly = data.groupby(['Store','Dept','Year','Quarter'])['Weekly_Sales'].sum().reset_index()

In [26]:
monthly.shape

(37266, 5)

In [27]:
year_2010 = monthly.loc[monthly['Year']==2010]

year_2011 = monthly.loc[monthly['Year']==2011]

year_2012 = monthly.loc[monthly['Year']==2012]

print(year_2010.shape,year_2011.shape,year_2012.shape)

year_2010.rename(columns={"Weekly_Sales": "Q_2010"},inplace=True)
year_2011.rename(columns={"Weekly_Sales": "Q_2011"},inplace=True)
year_2012.rename(columns={"Weekly_Sales": "Q_2012"},inplace=True)

(12362, 5) (12488, 5) (12416, 5)


/Users/dikshya/anaconda3/lib/python3.6/site-packages/pandas/core/frame.py:4238: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(**kwargs)


In [28]:
#year_2010.shape

(12362, 5)

In [29]:
year_2010.drop(columns='Year',inplace=True)
year_2011.drop(columns='Year',inplace=True)
year_2012.drop(columns='Year',inplace=True)

/Users/dikshya/anaconda3/lib/python3.6/site-packages/pandas/core/frame.py:4117: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [30]:
print(year_2010.shape,year_2011.shape,year_2012.shape)

(12362, 4) (12488, 4) (12416, 4)


In [31]:
year_2012 = year_2012.merge(year_2010,on=['Store','Dept','Quarter'],how='left')

In [32]:
year_2012 = year_2012.merge(year_2011,on=['Store','Dept','Quarter'],how='left')

In [33]:
year_2012.shape

(12416, 6)

In [34]:
year_2012.isna().sum()

Store        0
Dept         0
Quarter      0
Q_2012       0
Q_2010     435
Q_2011     280
dtype: int64

In [35]:
#for now
year_2012.fillna(0,inplace=True)

In [37]:
year_2012.columns

Index(['Store', 'Dept', 'Quarter', 'Q_2012', 'Q_2010', 'Q_2011'], dtype='object')

In [38]:
X = year_2012[['Store', 'Dept', 'Quarter',  'Q_2010', 'Q_2011']]
Y = year_2012[['Q_2012']]

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.20, random_state=42)

In [39]:
rfr = RandomForestRegressor(n_estimators = 250, random_state=0)               #RandomForestRegressor
rfr.fit(X_train,y_train)
y_pred=rfr.predict(X_test)

/Users/dikshya/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  


In [40]:
print(np.sqrt(metrics.mean_squared_error(y_test, y_pred)))
print(metrics.r2_score(y_test, y_pred))



25051.983265078397
0.9904359877668388


In [41]:
feature_importances = pd.DataFrame(rfr.feature_importances_,
                                   index = X_train.columns,
                                    columns=['importance']).sort_values('importance',ascending=False)

In [42]:
feature_importances

,importance
Q_2011,0.821492
Quarter,0.168830
Q_2010,0.004870
Dept,0.002965
Store,0.001843


In [43]:
X_test[:1]

,Store,Dept,Quarter,Q_2010,Q_2011
10000,36,20,4,604.46,254.44


In [47]:
test = pd.read_csv('Test_dataQ2.csv')

In [49]:
test[:3]

,Store,Dept,Quarter
0,1,1,4
1,1,2,4
2,1,3,4


In [53]:
year_2012[:2]

,Store,Dept,Quarter,Q_2012,Q_2010,Q_2011
0,1,1,1,319569.19,223200.22,277732.56
1,1,1,2,280039.64,283951.61,296287.24


In [50]:
test = test.merge(year_2010,on=['Store','Dept','Quarter'],how='left')

In [51]:
test = test.merge(year_2011,on=['Store','Dept','Quarter'],how='left')

In [52]:
test.shape

(3040, 5)

In [54]:
test[:2]

,Store,Dept,Quarter,Q_2010,Q_2011
0,1,1,4,400789.81,371856.41
1,1,2,4,634854.58,613442.89


In [56]:
test.fillna(0,inplace=True)

In [57]:
test['Q_2012'] = rfr.predict(test)

In [58]:
test[:2]

,Store,Dept,Quarter,Q_2010,Q_2011,Q_2012
0,1,1,4,400789.81,371856.41,102305.25104
1,1,2,4,634854.58,613442.89,173013.66608


In [59]:
test.to_csv('Q2_PredictionsQ42014.csv',index=False)

In [20]:
#monthly['Date'] = monthly['Year'].map(str)+'-0'+monthly['Month'].map(str)+'-01'

In [22]:
#monthly.to_csv('MonthlyData.csv',index=False)

In [60]:
year_2012.shape

(12416, 6)

In [61]:
test.shape

(3040, 6)

In [62]:
df = pd.concat([year_2012,test],axis=0)

/Users/dikshya/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """Entry point for launching an IPython kernel.


In [63]:
df.to_csv('Merged_Q2_allData.csv',index=False)